# TCC

In [2]:
import pandas as pd
import os as os
import fitz
import re

## Mapeando questões das provas

In [92]:
def map_questions(exam_string):    
    
    # Regex para o split usando "questão", espaço em branco e número (case-insensitive)
    pattern = re.compile(r'(questão\s+\d+)', re.IGNORECASE)
    result = pattern.split(exam_string)

    # Combina a identificação da questão com o conteúdo
    questions = []
    for i in range(2, len(result), 2):
        questions.append(result[i - 1] + ' ' + result[i])
    return questions

In [93]:
def separate_question(question):

    # Encontrar a correspondência na string
    alternatives_regex = r'\nA\s.*?\nB\s.*?\nC\s.*?\nD\s.*?\nE\s.*?\.'
    body = re.search(f'^(.*?)(?={alternatives_regex})', question, re.DOTALL).group(1)
    alternatives = re.search(f'({alternatives_regex})', question, re.DOTALL).group(1)
    
    return ({'body': body, 'alternatives':  alternatives})


In [80]:
testando_regex = """O termo agronegócio não se refere apenas à agricultura 
e à pecuária, pois as atividades ligadas a essa produção 
incluem fornecedores de equipamentos, serviços para a 
zona rural, industrialização e comercialização dos produtos.
Os dados seguintes refletem a participação percentual 
do agronegócio no PIB brasileiro entre os anos de 1998 
e 2008:
• 
1998: 21,33%;
• 
1999: 22,24%;
• 
2000: 22,87%;
• 
2001: 23,26%;
• 
2002: 25,31%;
• 
2003: 28,28%;
• 
2004: 27,79%;
• 
2005: 25,83%;
• 
2006: 23,92%;
• 
2007: 24,74%;
• 
2008: 26,46%.
Esses dados foram citados em uma palestra na 
qual o orador ressaltou uma queda da participação do 
agronegócio no PIB brasileiro e a posterior recuperação 
dessa participação, em termos percentuais.
De acordo com essas informações, o período de queda 
ocorreu entre os anos de 
A 1998 e 2001.
B 2001 e 2003.
C 2003 e 2006.
D 2003 e 2007.
E 2003 e 2008."""

testando_regex = """Questão 60          
a sociedade como um sistema justo de cooperação 
social consiste em uma das ideias familiares 
fundamentais, que dá estrutura e organização à justiça 
como equidade. A cooperação social guia-se por regras 
e procedimentos publicamente reconhecidos e aceitos 
por aqueles que cooperam como sendo apropriados para 
regular a sua conduta. Diz-se que a cooperação é justa 
porque seus termos são tais que todos os participantes 
podem razoavelmente aceitar, desde que todos os 
demais também o aceitem.
No contexto do pensamento político, a ideia apresentada mostra-se consoante o(a)
A  ideal republicano de governo. 
B  corrente tripartite dos poderes.
C  posicionamento crítico do socialismo.
D  legitimidade do absolutismo monárquico.
E  entendimento do contratualismo moderno."""

teste_2012 = """QUESTÃO 180 
Entre maratonistas, um parâmetro utilizado é o de economia de corrida (EC). O valor desse parâmetro é calculado 
pela razão entre o consumo de oxigênio, em mililitro (mL) por minuto (min), e a massa, em quilograma (kg), do atleta 
correndo a uma velocidade constante.
Um maratonista, visando melhorar sua performance, auxiliado por um médico, mensura o seu consumo de oxigênio 
por minuto a velocidade constante. Com base nesse consumo e na massa do atleta, o médico calcula o EC do atleta.
 A unidade de medida da grandeza descrita pelo parâmetro EC é
A fração com numerador min e denominador abre parêntese mL vezes kg fecha parêntese.
B fração com numerador mL e denominador abre parêntese min vezes kg fecha parêntese.
C fração com numerador abre parêntese min vezes mL fecha parêntese e denominador kg.
D fração com numerador abre parêntese min vezes kg fecha parêntese e denominador mL. 
E fração com numerador abre parêntese mL vezes kg fecha parêntese e denominador min."""
print(separate_question(teste_2012))

{'body': 'QUESTÃO 180 \nEntre maratonistas, um parâmetro utilizado é o de economia de corrida (EC). O valor desse parâmetro é calculado \npela razão entre o consumo de oxigênio, em mililitro (mL) por minuto (min), e a massa, em quilograma (kg), do atleta \ncorrendo a uma velocidade constante.\nUm maratonista, visando melhorar sua performance, auxiliado por um médico, mensura o seu consumo de oxigênio \npor minuto a velocidade constante. Com base nesse consumo e na massa do atleta, o médico calcula o EC do atleta.\n A unidade de medida da grandeza descrita pelo parâmetro EC é', 'alternatives': '\nA fração com numerador min e denominador abre parêntese mL vezes kg fecha parêntese.\nB fração com numerador mL e denominador abre parêntese min vezes kg fecha parêntese.\nC fração com numerador abre parêntese min vezes mL fecha parêntese e denominador kg.\nD fração com numerador abre parêntese min vezes kg fecha parêntese e denominador mL. \nE fração com numerador abre parêntese mL vezes kg fe

In [135]:
PATH = "../TCC/pdfs"
years = os.listdir(f'{PATH}')

### Convertendo PDF em string

In [136]:

for year in [2021]:
    
    exams = os.listdir(f'{PATH}/{year}')
    all_questions_edition = ''
    for test in exams:

        text = "" 
        if( "_D2_" in test):
            text += "\n------D2------\n"
        print(year)
        print(test)
        doc = fitz.open(f'{PATH}/{year}/{test}') 
        
        for page in doc: 
            if ("\nQuESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("\nQUESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("Questão" in page.get_text()):
                text += page.get_text()         
        all_questions_edition += text
       
    with open(f'{PATH}/{year}/prova{year}.txt', 'w', encoding='utf-8') as exam_str:
        exam_str.write(all_questions_edition)
        



2021
2021_PV_D1_CD9.pdf
2021
2021_PV_D2_CD11.pdf


### Lendo os arquivos com os textos das provas e mapeando as questões em um Dataframe

In [100]:
def map_questions_id(question_body):
    question_first_line = question_body.split('\n')[0]
    question_id = lambda x: ''.join([c for c in x if c.isnumeric()])
    return(question_id(question_first_line))

In [172]:
def agg_question_info(question, year,number):
    
    sp_area = None
    
    if(int(year) < 2017):
        if(number <= 45):
            area = 'CH - Ciências Humanas'
            day = 'D1'
        elif(number > 45 and number <= 90):
            area = 'CN - Ciências da Natureza'
            day = 'D1'
        elif(number > 90 and number <= 140):
            if(number > 90 and number <= 95):
                sp_area = 'Inglês'
            elif(number > 95 and number <= 100):
                sp_area = 'Espanhol'
            area = 'LC - Linguagens e Códigos'
            day = 'D2'
        elif(number > 140 and number <= 185):
            area = 'MT - Matemática e Códigos'
            day = 'D2'
    else:
        if(number <= 50):
            if(number >= 0 and number <= 5):
                sp_area = 'Inglês'
            elif(number > 5 and number <= 10):
                sp_area = 'Espanhol'
            area = 'LC - Linguagens e Códigos'
            day = 'D1'
        elif(number > 45 and number <= 90):
            area = 'CH - Ciências Humanas'
            day = 'D1'
        elif(number > 90 and number <= 140):
            area = 'CN - Ciências da Natureza'
            day = 'D2'
        elif(number > 140 and number <= 185):
            area = 'MT - Matemática e Códigos'
            day = 'D2'
        
    return pd.Series({
        'id': map_questions_id(question['body'])  + sp_area[0] if sp_area else map_questions_id(question['body']),
        'year': year,
        'area': area,
        'sp_area': sp_area,
        'day': day,
        'body': question['body'],
        'alternatives': question['alternatives'],
        
    })

#### Copiando o arquivo txt raw para o diretório de txts definitivo

In [90]:
TXT_PATH = "../TCC/tests_txt_files"

In [17]:
for year in years:
    with open(f'{PATH}/{year}/prova{year}.txt', 'r', encoding='utf-8') as test_txt:
        content = test_txt.read()
        
        with open(f'{TXT_PATH}/prova{year}.txt', 'w+', encoding='utf-8') as test_final_txt:
            test_final_txt.write(content)
        

#### Separando as questões de cada edição em uma tabela

In [173]:
teste = [2011,2012,2013,2015,2016]
exams_txt_files = os.listdir(f'{TXT_PATH}/')
for year in exams_txt_files:
    
    enem_questions = pd.DataFrame(columns=['id','year','area','sp_area','day','body','alternatives','answer'])
    with open(f'{TXT_PATH}/{year}/prova{year}.txt', 'r', encoding='utf-8') as exam_str:
        
        exam_string = exam_str.read()
        questions = map_questions(exam_string)
        
        print(len(questions))
        print(questions)
        print(year)
        for question_index in range(len(questions)):
            
            try:
            # separar os enunciados das alternativas    
                question_obj = separate_question(questions[question_index])
            except:
                print(questions[question_index])

            # construir Serie/linha da a ser colocada na tabela como infos da questão
            question_info = agg_question_info(question_obj, year, question_index + 1)
            
                # enem_questions = enem_questions.append(question_info, ignore_index=True)
            enem_questions = pd.concat([enem_questions, question_info.to_frame().transpose()], ignore_index=True)

    enem_questions.to_csv(f'../TCC/csvs/enem_{year}_questions.csv', index=False)
    
            


185
['QUESTÃO 01   \nNo mundo árabe, países governados há décadas por \nregimes políticos centralizadores contabilizam metade \nda população com menos de 30 anos; desses, 56% \ntêm acesso à internet. Sentindo-se sem perspectivas \nde futuro e diante da estagnação da economia, esses \njovens incubam vírus sedentos por modernidade e \ndemocracia. Em meados de dezembro, um tunisiano \nde 26 anos, vendedor de frutas, põe fogo no próprio \ncorpo em protesto por trabalho, justiça e liberdade. Uma \nsérie de manifestações eclode na Tunísia e, como uma \nepidemia, o vírus libertário começa a se espalhar pelos \npaíses vizinhos, derrubando em seguida o presidente \ndo Egito, Hosni Mubarak.  Sites e redes sociais – \ncomo o Facebook e o Twitter – ajudaram a mobilizar \nmanifestantes do norte da África a ilhas do Golfo Pérsico.\nConsiderando os movimentos políticos mencionados no \ntexto, o acesso à internet permitiu aos jovens árabes\nA reforçar a atuação dos regimes políticos existentes.\nB tom

## Converter os gabaritos para string e coletar os itens corretos, acrescentando na determinada tabela

In [85]:
ANSWERS = "../TCC/answers_pdfs"
CSV_PATH = "../TCC/csvs"

In [171]:
def map_answers(answers_key_txt):
    answers_key_lines = answers_key_txt.split('\n')
    
    keys = []
    for e in answers_key_lines:
        if(e and len(e.strip()) <= 3) or (e.strip() == "Anulado"):
            keys.append(e.strip())
            

    answers_map = {}
    right_answers = []
    keys.reverse()
        
    for item in keys:
        if(item.isnumeric()):
            if (int(item) < 10):
                item = '0' + item
            if(len(right_answers) > 1):
                right_answers.reverse()
                answers_map[item + "I"] = right_answers[0]
                answers_map[item + "E"] = right_answers[1]   
                right_answers = []
                continue
            answers_map[item] = right_answers[0] if len(right_answers) > 0 else 'Anulado'
            right_answers = []
        else:
            if(item == "Anulado"):
                right_answers.append(item)
            elif(item != ' ' and item != '' and item.isalpha()):
                right_answers.append(item)
            else:
                continue
    
             
    # Exibindo o resultado
    return answers_map

In [168]:
def aggregate_answers(answers_map, year):
    df = pd.read_csv(f'{CSV_PATH}/enem_{year}_questions.csv')
    df['answer'] = df['id'].map(answers_map)
    print(len(answers_map))
    
    return df

In [ ]:
teste = [2020]
all_years = os.listdir(f'{ANSWERS}')

for year in all_years:
    
    print(year)
    exams_editions = os.listdir(f'{ANSWERS}/{year}')
    all_questions_answers = ''
    for test_answers in exams_editions:
        
        if (("GABARITO" in test_answers) or ('GB' in test_answers)):
                
            doc = fitz.open(f'{ANSWERS}/{year}/{test_answers}') 
            exams_answers = ""

            for page in doc: 
                exams_answers += page.get_text()
        
            all_questions_answers += exams_answers
       
    with open(f'{ANSWERS}/{year}/test{year}_answers.txt', 'w', encoding='utf-8') as exam_str:
        exam_str.write(all_questions_answers)
        
    with open(f'{ANSWERS}/{year}/test{year}_answers.txt', 'r', encoding='utf-8') as exam_str:
        content = exam_str.read()
        answers_map = map_answers(content)
        df_with_answers = aggregate_answers(answers_map, year)
        df_with_answers.to_csv(f'../TCC/csvs/enem_{year}_questions.csv', index=False)